### openie from split file

In [1]:
from stanza.server import CoreNLPClient
import pickle
import pandas as pd
import spacy
from tqdm import tqdm

nlp = spacy.load('en_core_web_lg')
def prepare_text(text, client):
    # sentence segmentation
    doc=nlp(text) 
    result=[]
    
    # openie from every sentence
    for sent in list(doc.sents):
            sent=str(sent)
            sent = sent.rstrip()
            
            ann = client.annotate(sent,output_format='json')
            for sentence in ann['sentences']:
                for triple in sentence['openie']:
                    if triple:
                        result.append({'subject':triple['subject'],'relation': triple["relation"],'object':triple['object']})
    return result

def process(file_name):
    results = []
    # read file
    var = pickle.load(open('./cleaned_data_split/'+file_name+'.sav', 'rb'))
    
    # openie all
    with CoreNLPClient(be_quite=False, annotators=["openie"], memory='16G', endpoint='http://localhost:1234', threads=12) as client:
        for text in tqdm(var):
            results += prepare_text(text, client)
            
    # result save to csv
    result = pd.DataFrame(results)
    result.to_csv('./result_openie/'+file_name+'r.csv',index=False)

In [2]:
process('a0') # a1 a2 a3 ...

2021-10-07 02:05:44 INFO: Writing properties to tmp file: corenlp_server-8366b1862e134d9f.props
2021-10-07 02:05:44 INFO: Starting server with command: java -Xmx16G -cp C:\Users\pross\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 1234 -timeout 60000 -threads 12 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-8366b1862e134d9f.props -annotators openie -preload -outputFormat serialized
100%|██████████████████████████████████████████████████████████████████████████| 16042/16042 [2:05:53<00:00,  2.12it/s]
